In [3]:
import pickle
import gzip

active_members = pickle.load(gzip.open("../../Results/active_members.pickle.gz","rb"))

In [4]:
counts = {}

for board in active_members:
    counts[board] = len(active_members[board]['users'])
    
boards_sorted = sorted(counts.items(),key=lambda x: x[1],reverse=True)

In [12]:
for rank in [0,1,5,1000,1500,2000,4000,5000,10000,15000,20000,50000]:
    print(rank,boards_sorted[rank])

0 ('AskReddit', 1667185)
1 ('funny', 428512)
5 ('politics', 321808)
1000 ('dankchristianmemes', 6251)
1500 ('goodyearwelt', 4082)
2000 ('Sneks', 2911)
4000 ('vagabond', 1129)
5000 ('Garmin', 797)
10000 ('FirefoxCSS', 226)
15000 ('DailyDouble', 99)
20000 ('ASLinterpreters', 53)
50000 ('RoyaleConspiracy', 8)


In [6]:
relevant_subreddits = []
users = []

for board,_ in boards_sorted[0:50000]:
    relevant_subreddits.append(board)
    users.extend(active_members[board]['users'])
    
n = len(set(users))
n

8824747

In [7]:
import numpy as np
import math
from sklearn import preprocessing

def sets_common(a,b):
    return int(len(a.intersection(b)))

def h(x,n):
    return (x/float(n)) * math.log((x/float(n)))

def pmi(count_a,count_b,common,n):
    try:
        return math.log((common/float(n)) / float((count_a/n) * (count_b/n))) 
    except:
        return math.log((1/float(n)) / float((count_a/n) * (count_b/n)))

def calc_vectors(base_boards,desc):
    global active_members
    global n
    vectors = {}
    len_base_boards = len(base_boards)
    # number of authors in the top 50k subreddits

    for key,board in enumerate(reversed(relevant_subreddits[0:50000])):
        vectors[board] = np.zeros(len_base_boards) 
        for vec_key,base in enumerate(base_boards):
            common = sets_common(active_members[board]['users'],active_members[base]['users'])
            vectors[board][vec_key] = pmi(counts[board],counts[base],common,n)
        
        vectors[board] = preprocessing.normalize(vectors[board].reshape(1,-1), norm='l2')[0]

        logging.info("{} {}".format(key,desc))
            
    return vectors

In [8]:
hand_picked =  ['gaming', 'funny', 'Music', 'movies', 'television', 'sports', \
               'anime', 'comics', 'gifs', 'relationships', 'sex', 'Tinder', \
               'atheism', 'philosophy', 'The_Donald', 'worldnews', \
               'SandersForPresident', 'politics', 'PoliticalDiscussion',\
               'hillaryclinton', 'todayilearned', 'explainlikeimfive',\
               'books', 'science', 'askscience', 'Documentaries',\
               'history', 'pics', 'AdviceAnimals', 'photography',\
               'cats', 'Bitcoin', 'cscareerquestions', 'investing',\
               'learnprogramming', 'personalfinance', 'streetwear', 'Sneakers',\
               'Art', 'MakeupAddiction', 'malefashionadvice', 'Fitness', 'bicycling',\
               'Drugs', 'LSD', 'apple', 'Android', 'nvidia', 'nintendo', 'PS4', 'xboxone', \
               'Twitch', 'Vive', 'oculus', 'keto', 'vegan', 'food', 'Frugal', 'NoFap',\
               'depression', 'Anxiety', 'SuicideWatch']

In [16]:
from multiprocessing import Pool
import logging

logging.basicConfig(filename="output.log",level=logging.INFO)

def worker(desc):
    logging.info("Working on {}".format(desc))
    vectors = calc_vectors(base_boards[desc],desc)
    pickle_out = open("../../Results/" + desc + ".pickle","wb")
    pickle.dump(vectors, pickle_out)
    pickle_out.close()

base_boards = {
    'hand_picked': hand_picked,
    'politics': ['The_Donald','hillaryclinton','SandersForPresident'],
    'politics_alt': ['Republican','democrats','Liberal','Conservative','progressive', \
                     'SandersForPresident','The_Donald','hillaryclinton'],
    'politics_The_Donald': ['politics','The_Donald'],
    'sanders_The_Donald': ['SandersForPresident','The_Donald'],
    '0to100': relevant_subreddits[1:100],
    '0to250': relevant_subreddits[1:250],
    '0to500': relevant_subreddits[1:500],
    '0to2500': relevant_subreddits[1:2500],
    '2500to5000': relevant_subreddits[2500:5000],
    '0to5000': relevant_subreddits[1:5000],
    '0to10k': relevant_subreddits[1:10000],
    '0to25k': relevant_subreddits[1:25000],
    '0to50k': relevant_subreddits[1:50000],
#    '1kto10k': relevant_subreddits[1000:10000],
#    '0to100': relevant_subreddits[0:100],
#    '0to75': relevant_subreddits[0:75],
#    '0to150': relevant_subreddits[0:150],    
#   '0to175': relevant_subreddits[0:175],    
#     '0to200': relevant_subreddits[0:200],
#     '0to250': relevant_subreddits[0:250],
#     '0to300': relevant_subreddits[0:300],
#     '0to350': relevant_subreddits[0:350],
#     '0to400': relevant_subreddits[0:400],
#    '0to500': relevant_subreddits[0:500],
#    '0to600': relevant_subreddits[0:600],
#    '0to700': relevant_subreddits[0:700],
#    '0to800': relevant_subreddits[0:800],
#    '0to900': relevant_subreddits[0:900],
#    '0to1000': relevant_subreddits[0:1000],
#    '500to1000': relevant_subreddits[500:1000],
#    '1kto4k': relevant_subreddits[1000:4000],
#    '1kto3k': relevant_subreddits[1000:3000],
#    '1kto2k': relevant_subreddits[1000:2000],
#    '2kto4k': relevant_subreddits[2000:4000],
#    '2kto5k': relevant_subreddits[2000:5000],
#    '0to5k': relevant_subreddits[0:5000],
}

p = Pool(len(base_boards))
print(p.map(worker, base_boards))
p.close()

[None, None, None, None, None, None, None, None, None, None, None, None, None, None]
